### Dependencies

In [ ]:
# !pip install transformers
# !pip install datasets
# pip install yfinance

# !pip install llama-index-core==v0.11.16 llama-index-llms-together==0.2.0
# pip install langchain==0.3 langchain-together==0.2.0

In [ ]:
import os
import json
import datetime
import requests
import yfinance as yf
from typing import Dict

## Llama-index
from llama_index.llms.together import TogetherLLM
from llama_index.core.tools import FunctionTool
from llama_index.core.agent import ReActAgent
from llama_index.core.tools import FunctionTool

## LangChain
from langchain_core.tools import tool
from langchain_together import ChatTogether

### Agents 

#### Dataset

In [ ]:
stock = yf.Ticker("AAPL")
df = stock.history(period="1mo")
df[["Open", "Close"]].head(2)

In [ ]:
api_key = "" # ваш API ключ здесь!

In [ ]:
api_template = "https://newsapi.org/v2/everything?q={keyword}&apiKey={api_key}&from={date_from}"

articles = requests.get(
    api_template.format(
        keyword="Apple", 
        api_key=api_key,
        date_from=(datetime.datetime.today() - datetime.timedelta(days=30)).strftime('%Y-%m-%d')
    )
).json()

In [ ]:
def get_date_with_max_delta_price_for_ticker(ticker_name: str) -> Dict[str, str]:
    """receives ticker name, get dataframe with open and close prices of this ticker stock prices if 1 month period and returns dict with date of maximum daily difference in prices and max difference in prices itself"""
    stock = yf.Ticker(ticker_name)
    df = stock.history(period="1mo")
    df['delta'] = (df['Close'] - df['Open']).abs()
    id_max_delta = df['delta'].idxmax()
    return {
        'date': id_max_delta.strftime('%Y-%m-%d'), 
        'max_delta': df['delta'].max()
    }

def get_most_relative_news(
    company_name: str,
    date_with_max_price_delta: str
) -> str:
    """receives company name and string with date of max price delta, returns 5 most recent news articles about this company"""
    api_template = "https://newsapi.org/v2/everything?q={keyword}&apiKey={api_key}&from={date_from}"
    articles = requests.get(
        api_template.format(
            keyword=company_name, 
            api_key=api_key,
            date_from=(datetime.datetime.today() - datetime.timedelta(days=30)).strftime('%Y-%m-%d')
        )
    ).json()
    articles = [elem for elem in articles['articles'] if elem['title'] != '[Removed]' \
                and datetime.datetime.strptime(elem['publishedAt'], "%Y-%m-%dT%H:%M:%SZ") <= \
                    datetime.datetime.strptime(date_with_max_price_delta, '%Y-%m-%d')]
    top_five_news = sorted(articles, key=lambda x: datetime.datetime.strptime(x['publishedAt'], '%Y-%m-%dT%H:%M:%SZ'), reverse=True)[:5]
    top_five_news = [f'News article {i + 1}: ' + elem['content'] for i, elem in enumerate(top_five_news)]
    return '\n'.join(top_five_news)

## LlamaIndex

In [ ]:
model = "meta-llama/Meta-Llama-3.1-8B-Instruct-Turbo"
llm = TogetherLLM(model=model, api_key=os.environ['TOGETHER_API_KEY'])
get_date_with_max_delta_price_for_ticker_tool = FunctionTool.from_defaults(get_date_with_max_delta_price_for_ticker)
get_most_relative_news_tool = FunctionTool.from_defaults(get_most_relative_news)
agent = ReActAgent(
    llm=llm, 
    tools=[
        get_date_with_max_delta_price_for_ticker_tool, 
        get_most_relative_news_tool
    ], 
    memory=None, 
    verbose=True
)

In [ ]:
agent.query('Tell me when Apple stocks price difference was highest during last month and explain why it was so.')

## LangChain

In [ ]:
@tool
def get_date_with_max_delta_price_for_ticker_tool(ticker_name: str):
    """receives ticker name, get dataframe with open and close prices of this ticker stock prices if 1 month period and returns dict with date of maximum daily difference in prices and max difference in prices itself"""
    return get_date_with_max_delta_price_for_ticker(ticker_name)

@tool
def get_most_relative_news_tool(company_name: str, date_with_max_price_delta: str):
    """receives company name and string with date of max price delta, returns 5 most recent news articles about this company"""
    return get_most_relative_news(company_name, date_with_max_price_delta)

In [ ]:
FUNCTION_REGISTRY = {}

def parse_function_call(model_answer):
    model_answer = json.loads(model_answer)
    if model_answer['name'] in FUNCTION_REGISTRY.keys():
        return FUNCTION_REGISTRY[model_answer['name']](**model_answer['parameters'])
    
FUNCTION_REGISTRY['get_date_with_max_delta_price_for_ticker_tool'] = get_date_with_max_delta_price_for_ticker_tool
FUNCTION_REGISTRY['get_most_relative_news_tool'] = get_most_relative_news_tool

In [ ]:
lml = ChatTogether(api_key=os.environ["TOGETHER_API_KEY"], model=model)
llm = llm.bind_tools([get_date_with_max_delta_price_for_ticker_tool, get_most_relative_news_tool])

In [ ]:
messages = [
    {"role": "system", "content": "You are a helpful assistant"},
    {"role": "user", "content": "Tell me when Apple stocks price difference was highest during last month and explain why it was so."}
]
response = llm.invoke(messages)
messages.append({"role": "assistant", "content": response.content})

In [ ]:
FUNCTION_REGISTRY[response.tool_calls[0]['name']].invoke(response.tool_calls[0])